<a href="https://colab.research.google.com/github/katarina-ku/medulloblastoma/blob/main/Medulloblastoma_AI_Tumor_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MB-AI Tumor Segmentation


Note: To revert to the default mode, first delete the `nnUNetData/` folder from your Google Drive (https://drive.google.com/drive/u/0/my-drive). After doing so, re-run the process."

### Section 1: Prepare

Step 1 (Prepare): Please navigate to the '**Runtime**' option in your toolbar, select '**Change Runtime Type**', and then choose **a GPU** (for instance, **a T4 GPU**). This change is necessary to run the deep learning model.

Step 2 (Prepare): Install PyTorch and nnUNet-version1 (approximately 1 minute).

In [ ]:
!pip install torch
!git clone https://github.com/MIC-DKFZ/nnUNet.git
!cd nnUNet && git checkout nnunetv1 && pip install -e .

Step 3 (Prepare): Mount your google drive. Note: your google drive will only be accessible in your current Colab session, hosted by Google.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Step 4 (Prepare): Create nnUNet work directory and download the pretrained weights to your google drive (approximately 2 minutes).

In [ ]:
# create work directory necessary for running nnUNet
!cd /content/drive/MyDrive/ && mkdir -p ./nnUNetData/ ./nnUNetData/nnUNet_raw_data/ ./nnUNetData/nnUNet_preprocessed/

# download the pretrained weights of nnUNet to your google drive
!filename='/content/drive/MyDrive/nnUNetData/RESULTS_FOLDER.zip' && fileid='1cyy9LdzYCXD9PJbI2FIhuwzIeGj2Av0q' && \
 wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=${fileid}' -O- | sed -rn 's/.confirm=([0-9A-Za-z_]+)./\1\n/p')&id=${fileid}" -O ${filename} && rm -rf /tmp/cookies.txt

# decompress the zip folder and move everything to nnUNetData directory
!cd /content/drive/MyDrive/ && unzip ./nnUNetData/RESULTS_FOLDER.zip -d ./nnUNetData/ && rm -r ./nnUNetData/__MACOSX/ && rm ./nnUNetData/RESULTS_FOLDER.zip

Step 5 (Prepare): Download the five testing subjects and place them under the `testing_examples` directory in your google drive (approximately less than 1 minute).

In [ ]:
# create nnUNet_raw_data folder
!cd /content/drive/MyDrive/nnUNetData/nnUNet_raw_data/ && mkdir ./Task505_MedoNative/

# download the testing images (5 MB subjects with T1E and T2 nifti files)
!filename='/content/drive/MyDrive/nnUNetData/testing_examples.zip' && fileid='1eCQaBCdfb7xX-jzePx-t3NMH2FbUBsYi' && \
 wget --load-cookies /tmp/cookies.txt "https://drive.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=${fileid}' -O- | sed -rn 's/.confirm=([0-9A-Za-z_]+)./\1\n/p')&id=${fileid}" -O ${filename} && rm -rf /tmp/cookies.txt

# decompress the zip folder and move the testing subjects to nnUNet_raw_data directiory
!cd /content/drive/MyDrive/nnUNetData/ && unzip ./testing_examples.zip -d ./ && rm -r ./__MACOSX/ && rm ./testing_examples.zip

### Section 2: Inference

Segment the tumor area in the five test samples (approximately 5 minutes using a T4 GPU, your patience is appreciated).

In [ ]:
!export nnUNet_raw_data_base="/content/drive/MyDrive/nnUNetData" && \
 export nnUNet_preprocessed="/content/drive/MyDrive/nnUNetData/nnUNet_preprocessed" && \
 export RESULTS_FOLDER="/content/drive/MyDrive/nnUNetData/RESULTS_FOLDER" && \
 mkdir -p /content/drive/MyDrive/nnUNetData/predicted_segmentations/ && \
 nnUNet_predict -i /content/drive/MyDrive/nnUNetData/testing_examples/ \
                -o /content/drive/MyDrive/nnUNetData/predicted_segmentations/ -t 505 -m 3d_fullres

### Section 3: Visualization

Show the slice and segmentation that displays the largest tumor area.

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
examples_list = ['Native_001', 'Native_002', 'Native_003', 'Native_004', 'Native_005']
order2subtypes = {'001': 'WNT', '002': 'SHH TP53-', '003': 'SHH TP53+', '004': 'Group 3', '005': 'Group 4'}
images_dir = '/content/drive/MyDrive/nnUNetData/testing_examples/'
pred_labels_dir = '/content/drive/MyDrive/nnUNetData/predicted_segmentations/'


# for visualization, identify the slice that displays the largest tumor area
def find_max_slice(mask):
    sums = [np.sum(mask[..., i]) for i in range(mask.shape[-1])]
    return np.argmax(sums)


for example in examples_list:
    example_order = example.split('_')[-1]
    subtype_label = order2subtypes[example_order]
    t1e_image_path = os.path.join(images_dir, f'{example}_0000.nii.gz')
    t2_image_path = os.path.join(images_dir, f'{example}_0001.nii.gz')
    mask_path = os.path.join(pred_labels_dir, f'{example}.nii.gz')

    t1e_img = nib.load(t1e_image_path).get_fdata()
    t2_img = nib.load(t2_image_path).get_fdata()
    mask_img = nib.load(mask_path).get_fdata()

    max_slice = find_max_slice(mask_img)
    fig, ax = plt.subplots(1, 5, figsize=(20, 4))

    plt.figtext(0.1, 0.5, subtype_label, fontsize=12, va='center', ha='right', rotation='vertical')

    ax[0].imshow(t1e_img[..., max_slice], cmap='gray')
    ax[0].set_title(f'Subject {example_order} T1E')

    ax[1].imshow(t1e_img[..., max_slice], cmap='gray')
    ax[1].imshow(mask_img[..., max_slice], alpha=0.5, cmap='Reds')
    ax[1].set_title(f'Subject {example_order} T1E with segmentation')

    ax[2].imshow(t2_img[..., max_slice], cmap='gray')
    ax[2].set_title(f'Subject {example_order} T2')

    ax[3].imshow(t2_img[..., max_slice], cmap='gray')
    ax[3].imshow(mask_img[..., max_slice], alpha=0.5, cmap='Reds')
    ax[3].set_title(f'Subject {example_order} T2 with segmentation')

    ax[4].imshow(mask_img[..., max_slice], cmap='Reds')
    ax[4].set_title(f'Subject {example_order} Segmentation mask')

    for a in ax:
        a.axis('off')

    plt.show()


***Visualization***: To visualize the complete segmentation, you can download both the subject images and segmentation predictions to your local computer from specific directories in your Google Drive at https://drive.google.com/drive/u/0/my-drive. For subject images, navigate to `nnUNetData/testing_examples/`, and for the segmentation predictions, go to `nnUNetData/predicted_segmentations/`.

### Section 4: Run the model in your MRI Data

Step 1 (Run in your data): Create the directory for saving your MRI Data.

In [ ]:
!cd /content/drive/MyDrive/nnUNetData/ && mkdir ./your_testing_examples/

Step 2 (Run in your data): Upload your MRI data. Please follow the established naming conventions specified by the deep learning model.

Adhere to the **naming conventions** when preparing your files. Please name T1E images as Native_YourSubjectID_0000.nii.gz and T2 images as Native_YourSubjectID_0001.nii.gz. Once named correctly, upload these files to the nnUNetData/your_testing_examples directory in your Google Drive, accessible at https://drive.google.com/drive/u/0/my-drive. After completing the upload, you can proceed with the prediction process.



Step 3 (Run in your data): Predict the tumor segmentation in your data.

In [ ]:
!export nnUNet_raw_data_base="/content/drive/MyDrive/nnUNetData" && \
 export nnUNet_preprocessed="/content/drive/MyDrive/nnUNetData/nnUNet_preprocessed" && \
 export RESULTS_FOLDER="/content/drive/MyDrive/nnUNetData/RESULTS_FOLDER" && \
 mkdir -p /content/drive/MyDrive/nnUNetData/your_predicted_segmentations/ && \
 nnUNet_predict -i /content/drive/MyDrive/nnUNetData/your_testing_examples/ \
                -o /content/drive/MyDrive/nnUNetData/your_predicted_segmentations/ -t 505 -m 3d_fullres

Step 4 (Run in your data): Download the segmentation masks for reviewing.

You can access the predicted segmentation masks in the `nnUNetData/your_predicted_segmentations/` folder on Google Drive at https://drive.google.com/drive/u/0/my-drive.